In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(1_000_000_000_000) ## 1TB?

from calitp.tables import tbl
from calitp import query_sql
import calitp.magics
import branca

import shared_utils

from siuba import *
import pandas as pd
import geopandas as gpd
import shapely

import datetime as dt
import time
from zoneinfo import ZoneInfo

import rt_analysis as rt
import importlib

import gcsfs
fs = gcsfs.GCSFileSystem()

from tqdm import tqdm_notebook
from tqdm.notebook import trange, tqdm

/opt/conda/lib/python3.10/site-packages/geopandas/_compat.py:112: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
analysis_date = dt.date(2022, 6, 1)
itp_id = 159

In [3]:
pbar = tqdm()

0it [00:00, ?it/s]

In [4]:
rt_day = rt.OperatorDayAnalysis(itp_id, analysis_date, pbar)

found parquet
found parquet
found parquet
found parquet
found parquet
could not generate delays for trip t_1057276_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057308_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1439236_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057273_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1547204_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057313_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1057370_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1437743_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate delays for trip t_1437746_b_25985_tn_0
'NoneType' object has no attribute 'split'
could not generate de

In [27]:
importlib.reload(rt)
importlib.reload(shared_utils)

<module 'shared_utils' from '/home/jovyan/data-analyses/_shared_utils/shared_utils/__init__.py'>

In [7]:
rt.OperatorDayAnalysis._generate_stop_delay_view(rt_day)

AttributeError: 'NoneType' object has no attribute 'split'

In [22]:
rt_day.debug_dict['delays']

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id
0,783958,Lake Transit Yard,POINT (-226224.908 102987.490),p_4684,40447.291914,-8876945536004734190,t_1057276_b_25985_tn_0,18,None,2042,4,1
1,783997,Bowling Alley,POINT (-252624.574 117092.033),p_4684,3911.131886,-8876945536004734190,t_1057276_b_25985_tn_0,3,None,2042,4,1
2,784027,S Main & Peckham K-mart,POINT (-252195.365 116193.470),p_4684,5334.834153,-8876945536004734190,t_1057276_b_25985_tn_0,9,15:44:00,2042,4,1
3,784032,Social Security & DMV,POINT (-253004.033 117101.848),p_4684,2227.969017,-8876945536004734190,t_1057276_b_25985_tn_0,4,None,2042,4,1
4,784031,Parallel Dr. & Lakeport Blvd.,POINT (-252970.009 116911.875),p_4684,2015.223509,-8876945536004734190,t_1057276_b_25985_tn_0,5,15:39:00,2042,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...
1347,783967,Coyote Valley Medical Plaza,POINT (-221839.439 89567.738),p_4694,21972.957069,-3413408852418918973,t_1057260_b_25985_tn_0,7,None,2033,3,0
1348,783969,Twin Lakes,POINT (-226483.235 99162.859),p_4694,6660.729957,-3413408852418918973,t_1057260_b_25985_tn_0,5,None,2033,3,0
1349,784009,Job Zone,POINT (-226266.582 103233.035),p_4694,1894.496860,-3413408852418918973,t_1057260_b_25985_tn_0,2,None,2033,3,0
1350,784460,Charter School,POINT (-226978.746 84861.591),p_4694,30094.672828,-3413408852418918973,t_1057260_b_25985_tn_0,9,None,2033,3,0


In [25]:
df = rt_day.debug_dict['delays'] >> filter(_.trip_id == 't_1057276_b_25985_tn_0') >> arrange(_.stop_sequence)

In [29]:
def fix_arrival_time(gtfs_timestring: str) -> tuple[str, int]:
    """Reformats a GTFS timestamp (which allows the hour to exceed
    24 to mark service day continuity)
    to standard 24-hour time.
    """
    extra_day = 0
    if not gtfs_timestring: # preserve none if time not provided
        return None, extra_day
    split = gtfs_timestring.split(":")
    hour = int(split[0])

    if hour >= 24:
        extra_day = 1
        split[0] = str(hour - 24)
        corrected = (":").join(split)
        return corrected.strip(), extra_day

    else:
        return gtfs_timestring.strip(), extra_day

In [30]:
df.arrival_time = df.arrival_time.map(lambda x: fix_arrival_time(x)[0])

In [31]:
df

,stop_id,stop_name,geometry,shape_id,shape_meters,trip_key,trip_id,stop_sequence,arrival_time,route_id,route_short_name,direction_id
7,783995,Main & Third,POINT (-252059.032 118071.269),p_4684,0.134045,-8876945536004734190,t_1057276_b_25985_tn_0,1,15:30:00,2042,4,1
9,784033,Main & D St.,POINT (-252082.645 117466.756),p_4684,604.994549,-8876945536004734190,t_1057276_b_25985_tn_0,2,15:33:00,2042,4,1
1,783997,Bowling Alley,POINT (-252624.574 117092.033),p_4684,3911.131886,-8876945536004734190,t_1057276_b_25985_tn_0,3,None,2042,4,1
3,784032,Social Security & DMV,POINT (-253004.033 117101.848),p_4684,2227.969017,-8876945536004734190,t_1057276_b_25985_tn_0,4,None,2042,4,1
4,784031,Parallel Dr. & Lakeport Blvd.,POINT (-252970.009 116911.875),p_4684,2015.223509,-8876945536004734190,t_1057276_b_25985_tn_0,5,15:39:00,2042,4,1
17,784030,Bruno's (between driveways),POINT (-252412.486 117067.151),p_4684,1299.447072,-8876945536004734190,t_1057276_b_25985_tn_0,6,None,2042,4,1
8,784029,Grocery Outlet,POINT (-252142.200 117010.238),p_4684,4516.515981,-8876945536004734190,t_1057276_b_25985_tn_0,7,None,2042,4,1
15,784028,Hospice Thrift Store,POINT (-252181.314 116593.900),p_4684,4934.032066,-8876945536004734190,t_1057276_b_25985_tn_0,8,None,2042,4,1
2,784027,S Main & Peckham K-mart,POINT (-252195.365 116193.470),p_4684,5334.834153,-8876945536004734190,t_1057276_b_25985_tn_0,9,15:44:00,2042,4,1
13,784026,By Rotten Robbies,POINT (-251935.398 115304.243),p_4684,6264.761484,-8876945536004734190,t_1057276_b_25985_tn_0,10,None,2042,4,1


In [34]:
df['arrival_time'] = df.apply(lambda x:
                    dt.datetime.combine(analysis_date,
                                        dt.datetime.strptime(x.arrival_time, '%H:%M:%S').time()) if x.arrival_time else None,
                                        axis = 1) ## format scheduled arrival times

In [52]:
df.arrival_time.iloc[0]

Timestamp('2022-06-01 15:30:00')

In [ ]:
## TODO timezone convert back and forth to enable interpolation, review other time handling code

In [53]:
dt.timezone.

datetime.timezone

In [48]:
df.arrival_time.iloc[0].timestamp()

1654097400.0

In [49]:
# set system time
os.environ["TZ"] = "America/Los_Angeles"
time.tzset()

In [50]:
dt.datetime.fromtimestamp(1654097400.0)

datetime.datetime(2022, 6, 1, 8, 30)

In [39]:
df.arrival_time.interpolate(method='time')

ValueError: Invalid fill method. Expecting pad (ffill) or backfill (bfill). Got time

In [ ]:
df